## Goal: test multi-objective optimization

### 2 ring multi-objective: successful optimization for field strength!

In [40]:
import numpy as np
from scipy.optimize import minimize, Bounds
from maghelper import get_field_on_axes, get_nonuniformity, make_flux_stream, centered_sweep_range, get_grid_nonuniformity, make_grid
from maggeometry import two_rings
import magpylib as magpy

x_base = [150, 10]
field_strength_target = 10 #Gauss

Br = 1.09e3
grid_res = 101
grid = make_grid([-10, 10], [-10, 10], grid_res)
def nonuniformity(x):
    innerrad, width = x_base
    thickness, dist = x
    magnets = two_rings(Br, innerrad, width, thickness, dist)
    grid_field, center_field, av_nonuniformity, max_abs_nonuniformity = get_grid_nonuniformity(magnets, grid, grid_res)
    return av_nonuniformity

def strength_and_nonuniformity(x):
    innerrad, width = x_base
    thickness, dist = x
    magnets = two_rings(Br, innerrad, width, thickness, dist)
    grid_field, center_field, av_nonuniformity, max_abs_nonuniformity = get_grid_nonuniformity(magnets, grid, grid_res)
    cost = np.log10(av_nonuniformity) + abs(center_field - field_strength_target)
    return cost

# innerrad, width, thickness, dist
bounds = ((2, 20), (1, 300))
x0 = [2, 2.62805985e+02]
options = {'disp':True, 'fatol': 5e-17}
# options = {'disp':True}
method = "Nelder-Mead"
res = minimize(strength_and_nonuniformity, x0, method=method, options=options, bounds=bounds)
# res = minimize(nonuniformity, x0, method=method, options=options, bounds=bounds)
print(res)
print(res.x)

Optimization terminated successfully.
         Current function value: -6.043279
         Iterations: 150
         Function evaluations: 319
       message: Optimization terminated successfully.
       success: True
        status: 0
           fun: -6.043279411155146
             x: [ 1.370e+01  2.625e+02]
           nit: 150
          nfev: 319
 final_simplex: (array([[ 1.370e+01,  2.625e+02],
                       [ 1.370e+01,  2.625e+02],
                       [ 1.370e+01,  2.625e+02]]), array([-6.043e+00, -6.043e+00, -6.043e+00]))
[ 13.70088584 262.53186908]


In [41]:
print(nonuniformity(res.x))
innerrad, width = x_base
thickness, dist = res.x
magnets = two_rings(Br, innerrad, width, thickness, dist)
grid_field, center_field, av_nonuniformity, max_abs_nonuniformity = get_grid_nonuniformity(magnets, grid, grid_res)
print(center_field)

9.05150052212399e-07
10.00000000804187


### 2 rings w/ cost function v2

In [46]:
import numpy as np
from scipy.optimize import minimize, Bounds
from maghelper import get_field_on_axes, get_nonuniformity, make_flux_stream, centered_sweep_range, get_grid_nonuniformity, make_grid
from maggeometry import two_rings
import magpylib as magpy

field_strength_target = 10 #Gauss

Br = 1.09e3
grid_res = 101
grid = make_grid([-10, 10], [-10, 10], grid_res)
def nonuniformity(x):
    innerrad, width, thickness, dist = x
    magnets = two_rings(Br, innerrad, width, thickness, dist)
    grid_field, center_field, av_nonuniformity, max_abs_nonuniformity = get_grid_nonuniformity(magnets, grid, grid_res)
    return av_nonuniformity

def strength_and_nonuniformity2(x):
    innerrad, width, thickness, dist = x
    magnets = two_rings(Br, innerrad, width, thickness, dist)
    grid_field, center_field, av_nonuniformity, max_abs_nonuniformity = get_grid_nonuniformity(magnets, grid, grid_res)
    cost = av_nonuniformity * abs(center_field - field_strength_target)
    return cost

# innerrad, width, thickness, dist
bounds = ((1, 150), (2, 20), (2, 20), (1, 300))
x0 = [150, 10, 2, 2.62805985e+02]
options = {'disp':True, 'fatol': 5e-17}
# options = {'disp':True}
method = "Nelder-Mead"
res = minimize(strength_and_nonuniformity2, x0, method=method, options=options, bounds=bounds)
# res = minimize(nonuniformity, x0, method=method, options=options, bounds=bounds)
print(res)
print(res.x)

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 198
         Function evaluations: 355
       message: Optimization terminated successfully.
       success: True
        status: 0
           fun: 5.693968770624182e-17
             x: [ 1.500e+02  1.988e+01  7.331e+00  2.705e+02]
           nit: 198
          nfev: 355
 final_simplex: (array([[ 1.500e+02,  1.988e+01,  7.331e+00,  2.705e+02],
                       [ 1.500e+02,  1.988e+01,  7.331e+00,  2.705e+02],
                       ...,
                       [ 1.500e+02,  1.988e+01,  7.331e+00,  2.705e+02],
                       [ 1.500e+02,  1.988e+01,  7.331e+00,  2.705e+02]]), array([ 5.694e-17,  6.801e-17,  7.969e-17,  8.279e-17,
                        8.708e-17]))
[150.          19.87685011   7.33075237 270.48619207]


In [48]:
print(nonuniformity(res.x))
innerrad, width, thickness, dist = res.x
magnets = two_rings(Br, innerrad, width, thickness, dist)
grid_field, center_field, av_nonuniformity, max_abs_nonuniformity = get_grid_nonuniformity(magnets, grid, grid_res)
print(center_field)

7.98858430954589e-07
10.000000000071276


### 3 ring multiobjective test

In [32]:
import numpy as np
from scipy.optimize import minimize, Bounds
from maghelper import get_field_on_axes, get_nonuniformity, make_flux_stream, centered_sweep_range, get_grid_nonuniformity, make_grid
from maggeometry import two_rings, three_rings
import magpylib as magpy

x0 = [1.50000000e+02, 1.39848097e+02, 1.99851214e+01, 1.99624362e+01, 9.00538433e+01]
field_strength_target = 10 #Gauss

Br = 1.09e3
grid_res = 101
grid = make_grid([-10, 10], [-10, 10], grid_res)

def nonuniformity(x):
    innerrad1, innerrad2, width, thickness, dist = x
    magnets = three_rings(Br, innerrad1, innerrad2, width, thickness, dist)
    grid_field, center_field, av_nonuniformity, max_abs_nonuniformity = get_grid_nonuniformity(magnets, grid, grid_res)
    cost = np.log10(av_nonuniformity) + abs(center_field - field_strength_target)
    return av_nonuniformity

def strength_and_nonuniformity(x):
    innerrad1, innerrad2, width, thickness, dist = x
    magnets = three_rings(Br, innerrad1, innerrad2, width, thickness, dist)
    grid_field, center_field, av_nonuniformity, max_abs_nonuniformity = get_grid_nonuniformity(magnets, grid, grid_res)
    cost = np.log10(av_nonuniformity) + abs(center_field - field_strength_target)
    return cost
bounds = ((1, 150), (1, 150), (2, 20), (2, 20), (1, 300))
options = {'disp':True, 'fatol': 5e-17}
method = "Nelder-Mead"
res = minimize(strength_and_nonuniformity, x0, method=method, options=options, bounds=bounds)
print(res)
print(res.x)

       message: Maximum number of function evaluations has been exceeded.
       success: False
        status: 1
           fun: -3.690908626260408
             x: [ 1.500e+02  9.797e+01  1.974e+01  2.000e+01  1.256e+02]
           nit: 550
          nfev: 1000
 final_simplex: (array([[ 1.500e+02,  9.797e+01, ...,  2.000e+01,
                         1.256e+02],
                       [ 1.500e+02,  9.797e+01, ...,  2.000e+01,
                         1.256e+02],
                       ...,
                       [ 1.500e+02,  9.797e+01, ...,  2.000e+01,
                         1.256e+02],
                       [ 1.500e+02,  9.797e+01, ...,  2.000e+01,
                         1.256e+02]]), array([-3.691e+00, -3.691e+00, -3.691e+00, -3.691e+00,
                       -3.691e+00, -3.691e+00]))
[150.          97.97223407  19.74079492  19.9999088  125.63956837]


/var/folders/06/7pjfbw510cz2ch1m7lx8vvl80000gn/T/ipykernel_48807/1068926402.py:31: RuntimeWarning: Maximum number of function evaluations has been exceeded.
  res = minimize(strength_and_nonuniformity, x0, method=method, options=options, bounds=bounds)


In [33]:
print(nonuniformity(res.x))
innerrad1, innerrad2, width, thickness, dist = res.x
magnets = three_rings(Br, innerrad1, innerrad2, width, thickness, dist)
grid_field, center_field, av_nonuniformity, max_abs_nonuniformity = get_grid_nonuniformity(magnets, grid, grid_res)
print(center_field)

0.00020374707078497887
9.999999999986215


In [36]:
import numpy as np
from scipy.optimize import minimize, Bounds
from maghelper import get_field_on_axes, get_nonuniformity, make_flux_stream, centered_sweep_range, get_grid_nonuniformity, make_grid
from maggeometry import two_rings, three_rings
import magpylib as magpy

x0 = [1.50000000e+02, 1.39848097e+02, 1.99851214e+01, 1.99624362e+01, 9.00538433e+01]
field_strength_target = 10 #Gauss

Br = 1.09e3
grid_res = 101
grid = make_grid([-10, 10], [-10, 10], grid_res)

def nonuniformity(x):
    innerrad1, innerrad2, width, thickness, dist = x
    magnets = three_rings(Br, innerrad1, innerrad2, width, thickness, dist)
    grid_field, center_field, av_nonuniformity, max_abs_nonuniformity = get_grid_nonuniformity(magnets, grid, grid_res)
    cost = np.log10(av_nonuniformity) + abs(center_field - field_strength_target)
    return av_nonuniformity

def strength_and_nonuniformity(x):
    innerrad1, innerrad2, width, thickness, dist = x
    magnets = three_rings(Br, innerrad1, innerrad2, width, thickness, dist)
    grid_field, center_field, av_nonuniformity, max_abs_nonuniformity = get_grid_nonuniformity(magnets, grid, grid_res)
    cost = np.log10(av_nonuniformity) + abs(center_field - field_strength_target)
    return cost

def strength_and_nonuniformity2(x):
    innerrad1, innerrad2, width, thickness, dist = x
    magnets = three_rings(Br, innerrad1, innerrad2, width, thickness, dist)
    grid_field, center_field, av_nonuniformity, max_abs_nonuniformity = get_grid_nonuniformity(magnets, grid, grid_res)
    cost = av_nonuniformity*abs(center_field - field_strength_target)
    return cost

bounds = ((1, 150), (1, 150), (2, 20), (2, 20), (1, 300))
options = {'disp':True, 'fatol': 5e-17}
method = "Nelder-Mead"
res = minimize(strength_and_nonuniformity2, x0, method=method, options=options, bounds=bounds)
print(res)
print(res.x)

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 471
         Function evaluations: 817
       message: Optimization terminated successfully.
       success: True
        status: 0
           fun: 6.372310428386734e-17
             x: [ 1.500e+02  1.398e+02  1.771e+01  2.000e+00  8.864e+01]
           nit: 471
          nfev: 817
 final_simplex: (array([[ 1.500e+02,  1.398e+02, ...,  2.000e+00,
                         8.864e+01],
                       [ 1.500e+02,  1.398e+02, ...,  2.000e+00,
                         8.864e+01],
                       ...,
                       [ 1.500e+02,  1.398e+02, ...,  2.000e+00,
                         8.864e+01],
                       [ 1.500e+02,  1.398e+02, ...,  2.000e+00,
                         8.864e+01]]), array([ 6.372e-17,  7.634e-17,  7.984e-17,  8.629e-17,
                        9.541e-17,  1.042e-16]))
[150.         139.79143431  17.70797611   2.          88.63927975]


In [37]:
print(nonuniformity(res.x))
innerrad1, innerrad2, width, thickness, dist = res.x
magnets = three_rings(Br, innerrad1, innerrad2, width, thickness, dist)
grid_field, center_field, av_nonuniformity, max_abs_nonuniformity = get_grid_nonuniformity(magnets, grid, grid_res)
print(center_field)

6.201987965018676e-08
9.999999998972537


### 4 ring multiobjective test

In [34]:
import numpy as np
from scipy.optimize import minimize, Bounds
from maghelper import get_field_on_axes, get_nonuniformity, make_flux_stream, centered_sweep_range, get_grid_nonuniformity, make_grid
from maggeometry import four_rings
import magpylib as magpy
from magpylib.magnet import CylinderSegment
from alive_progress import alive_bar

Br = 1.09e3
grid_res = 101
grid = make_grid([-10, 10], [-10, 10], grid_res)

# cost function
def nonuniformity(x):
    innerrad1, innerrad2, width, thickness, dist1, dist2 = x
    magnets = four_rings(Br, innerrad1, innerrad2, width, thickness, dist1, dist2)
    grid_field, center_field, av_nonuniformity, max_abs_nonuniformity = get_grid_nonuniformity(magnets, grid, grid_res)
    return av_nonuniformity

def strength_and_nonuniformity(x):
    innerrad1, innerrad2, width, thickness, dist1, dist2 = x
    magnets = four_rings(Br, innerrad1, innerrad2, width, thickness, dist1, dist2)
    grid_field, center_field, av_nonuniformity, max_abs_nonuniformity = get_grid_nonuniformity(magnets, grid, grid_res)
    cost = np.log10(av_nonuniformity) + abs(center_field - field_strength_target)
    return cost

# innerrad, width, thickness, dist
# bounds = ([1, 2, 2, 1], [150, 10, 20, 300])
bounds = ((1, 150), (1, 150), (2, 20), (2, 20), (1, 300), (1, 300))
options = {'disp':True, 'fatol': 5e-17}
method = "Nelder-Mead"
x0 = [144.38226051, 51.11048466, 19.94272085, 2, 155.70613803, 183.10542199]

res = minimize(strength_and_nonuniformity, x0, method=method, options=options, bounds=bounds)
# res = minimize(nonuniformity, x0, method=method, options=options, bounds=bounds)
print(res)
print(res.x)

Optimization terminated successfully.
         Current function value: -8.973462
         Iterations: 288
         Function evaluations: 674
       message: Optimization terminated successfully.
       success: True
        status: 0
           fun: -8.97346230620897
             x: [ 1.444e+02  5.111e+01  1.994e+01  2.102e+00  1.557e+02
                  1.831e+02]
           nit: 288
          nfev: 674
 final_simplex: (array([[ 1.444e+02,  5.111e+01, ...,  1.557e+02,
                         1.831e+02],
                       [ 1.444e+02,  5.111e+01, ...,  1.557e+02,
                         1.831e+02],
                       ...,
                       [ 1.444e+02,  5.111e+01, ...,  1.557e+02,
                         1.831e+02],
                       [ 1.444e+02,  5.111e+01, ...,  1.557e+02,
                         1.831e+02]]), array([-8.973e+00, -8.973e+00, -8.973e+00, -8.973e+00,
                       -8.973e+00, -8.973e+00, -8.973e+00]))
[144.3822043   51.1104813   19.94298

In [35]:
print(nonuniformity(res.x))
innerrad1, innerrad2, width, thickness, dist1, dist2 = res.x
magnets = four_rings(Br, innerrad1, innerrad2, width, thickness, dist1, dist2)
grid_field, center_field, av_nonuniformity, max_abs_nonuniformity = get_grid_nonuniformity(magnets, grid, grid_res)
print(center_field)

1.0630078269114723e-09
9.999998768448117


### 4 rings using version 2 of cost function

In [38]:
import numpy as np
from scipy.optimize import minimize, Bounds
from maghelper import get_field_on_axes, get_nonuniformity, make_flux_stream, centered_sweep_range, get_grid_nonuniformity, make_grid
from maggeometry import four_rings
import magpylib as magpy
from magpylib.magnet import CylinderSegment
from alive_progress import alive_bar

Br = 1.09e3
grid_res = 101
grid = make_grid([-10, 10], [-10, 10], grid_res)

# cost function
def nonuniformity(x):
    innerrad1, innerrad2, width, thickness, dist1, dist2 = x
    magnets = four_rings(Br, innerrad1, innerrad2, width, thickness, dist1, dist2)
    grid_field, center_field, av_nonuniformity, max_abs_nonuniformity = get_grid_nonuniformity(magnets, grid, grid_res)
    return av_nonuniformity

def strength_and_nonuniformity(x):
    innerrad1, innerrad2, width, thickness, dist1, dist2 = x
    magnets = four_rings(Br, innerrad1, innerrad2, width, thickness, dist1, dist2)
    grid_field, center_field, av_nonuniformity, max_abs_nonuniformity = get_grid_nonuniformity(magnets, grid, grid_res)
    cost = np.log10(av_nonuniformity) + abs(center_field - field_strength_target)
    return cost

def strength_and_nonuniformity2(x):
    innerrad1, innerrad2, width, thickness, dist1, dist2 = x
    magnets = four_rings(Br, innerrad1, innerrad2, width, thickness, dist1, dist2)
    grid_field, center_field, av_nonuniformity, max_abs_nonuniformity = get_grid_nonuniformity(magnets, grid, grid_res)
    cost = av_nonuniformity*abs(center_field - field_strength_target)
    return cost
# innerrad, width, thickness, dist
# bounds = ([1, 2, 2, 1], [150, 10, 20, 300])
bounds = ((1, 150), (1, 150), (2, 20), (2, 20), (1, 300), (1, 300))
options = {'disp':True, 'fatol': 5e-17}
method = "Nelder-Mead"
x0 = [144.38226051, 51.11048466, 19.94272085, 2, 155.70613803, 183.10542199]

res = minimize(strength_and_nonuniformity2, x0, method=method, options=options, bounds=bounds)
print(res)
print(res.x)

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 157
         Function evaluations: 310
       message: Optimization terminated successfully.
       success: True
        status: 0
           fun: 9.48486231110599e-17
             x: [ 1.444e+02  5.111e+01  1.994e+01  2.102e+00  1.557e+02
                  1.831e+02]
           nit: 157
          nfev: 310
 final_simplex: (array([[ 1.444e+02,  5.111e+01, ...,  1.557e+02,
                         1.831e+02],
                       [ 1.444e+02,  5.111e+01, ...,  1.557e+02,
                         1.831e+02],
                       ...,
                       [ 1.444e+02,  5.111e+01, ...,  1.557e+02,
                         1.831e+02],
                       [ 1.444e+02,  5.111e+01, ...,  1.557e+02,
                         1.831e+02]]), array([ 9.485e-17,  1.018e-16,  1.091e-16,  1.192e-16,
                        1.279e-16,  1.393e-16,  1.403e-16]))
[144.38058208  51.11036141  19.942

In [39]:
print(nonuniformity(res.x))
innerrad1, innerrad2, width, thickness, dist1, dist2 = res.x
magnets = four_rings(Br, innerrad1, innerrad2, width, thickness, dist1, dist2)
grid_field, center_field, av_nonuniformity, max_abs_nonuniformity = get_grid_nonuniformity(magnets, grid, grid_res)
print(center_field)

1.0937169158220585e-09
10.000000086721364
